In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

# Define the input folder where preprocessed images are located
input_folder = "/content/drive/MyDrive/preprocessing_multiprocessingmodule"

# Define the output CSV file path for storing results
result_csv_path = "/content/drive/MyDrive/imgpath_gvi.csv"

def calculate_green_percentage(image_path):
    image = cv2.imread(image_path)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_green = np.array([40, 40, 40])
    upper_green = np.array([80, 255, 255])
    green_mask = cv2.inRange(hsv_image, lower_green, upper_green)

    # Calculate the green percentage with two decimal places
    total_pixels = image.shape[0] * image.shape[1]
    green_pixels = np.sum(green_mask > 0)
    green_percentage = round((green_pixels / total_pixels) , 4)

    return image_path, green_percentage  # Return the full image path

# Check if the result CSV file exists and remove it
if os.path.exists(result_csv_path):
    os.remove(result_csv_path)

# List all preprocessed image files
image_files = [os.path.join(input_folder, filename) for filename in os.listdir(input_folder) if filename.endswith(".jpg")]

# Calculate green percentage for each preprocessed image
results = []
for image_path in image_files:
    image_path, green_percentage = calculate_green_percentage(image_path)
    results.append([image_path, green_percentage])  # Include full image path

# Save the results to a new CSV file
df = pd.DataFrame(results, columns=['Image', 'Green Percentage'])  # Update column name
df.to_csv(result_csv_path, index=False)

print("Green percentage calculation complete.")
print(f"New results saved to {result_csv_path}")


Green percentage calculation complete.
New results saved to /content/drive/MyDrive/imgpath_gvi.csv


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

# Define the input CSV file containing image paths and green percentages
csv_file_path = "/content/drive/MyDrive/imgpath_gvi.csv"

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Define the input folder where images are located
input_folder = "/content/drive/MyDrive/preprocessing_multiprocessingmodule"

# Load and preprocess images while resizing to a consistent size (e.g., 224x224)
def load_and_preprocess_image(image_path):
    img = cv2.imread(os.path.join(input_folder, image_path))
    img = cv2.resize(img, (224, 224))
    img = img / 255.0  # Normalize pixel values to [0, 1]
    return img

# Define a generator to load and preprocess images in batches
def image_data_generator(df, batch_size):
    while True:
        for i in range(0, len(df), batch_size):
            batch_df = df.iloc[i:i+batch_size]
            batch_images = [load_and_preprocess_image(image_path) for image_path in batch_df['Image']]
            batch_X = np.array(batch_images)
            batch_y = batch_df['Green Percentage'].values
            yield batch_X, batch_y




In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['Image'], df['Green Percentage'], test_size=0.2, random_state=42)

# Create the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')  # Linear activation for regression
])


In [ ]:

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])



In [ ]:
# Define batch size
batch_size = 32

# Train the model using the generator
steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_test) // batch_size
epochs = 10



In [ ]:
train_generator = image_data_generator(pd.DataFrame({'Image': X_train, 'Green Percentage': y_train}), batch_size)
test_generator = image_data_generator(pd.DataFrame({'Image': X_test, 'Green Percentage': y_test}), batch_size)



In [ ]:
history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=epochs,
                    validation_data=test_generator, validation_steps=validation_steps)



Epoch 1/10
127/127 [==============================] - 417s 3s/step - loss: 0.7391 - mae: 0.3616 - val_loss: 0.1506 - val_mae: 0.2826
Epoch 2/10
127/127 [==============================] - 440s 3s/step - loss: 0.1392 - mae: 0.2786 - val_loss: 0.1242 - val_mae: 0.2664
Epoch 3/10
127/127 [==============================] - 434s 3s/step - loss: 0.1147 - mae: 0.2627 - val_loss: 0.1028 - val_mae: 0.2511
Epoch 4/10
127/127 [==============================] - 422s 3s/step - loss: 0.0938 - mae: 0.2364 - val_loss: 0.0818 - val_mae: 0.2194
Epoch 5/10
127/127 [==============================] - 424s 3s/step - loss: 0.0788 - mae: 0.2148 - val_loss: 0.0677 - val_mae: 0.2031
Epoch 6/10
127/127 [==============================] - 440s 3s/step - loss: 0.0688 - mae: 0.2017 - val_loss: 0.0586 - val_mae: 0.1952
Epoch 7/10
127/127 [==============================] - 440s 3s/step - loss: 0.0639 - mae: 0.1960 - val_loss: 0.0527 - val_mae: 0.1892
Epoch 8/10
127/127 [==============================] - 416s 3s/step - 

In [ ]:
loss, mae = model.evaluate(test_generator, steps=validation_steps)
print(f"Mean Absolute Error (MAE): {mae}")


31/31 [==============================] - 29s 919ms/step - loss: 0.0453 - mae: 0.1812
Mean Absolute Error (MAE): 0.18120834231376648


In [ ]:
# Specify the path to save the trained model
model_save_path = "/content/drive/MyDrive/cnn_trained_model.h5"

# Save the trained model
model.save(model_save_path)


In [2]:
import cv2
import numpy as np

# Define the lower and upper HSV thresholds for green
lower_green = np.array([40, 40, 40])
upper_green = np.array([80, 255, 255])

# Define a function to preprocess the new image
def preprocess_image(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Create a mask for the green regions
    green_mask = cv2.inRange(hsv_image, lower_green, upper_green)

    # Calculate the green percentage
    total_pixels = image.shape[0] * image.shape[1]
    green_pixels = np.sum(green_mask > 0)
    green_percentage = (green_pixels / total_pixels) * 100.0

    return green_percentage


In [ ]:
# Define the path to the new image you want to analyze
new_image_path = "/content/drive/MyDrive/images/3484_sat_34.jpg"

# Calculate the green percentage of the new image
predicted_green_percentage = preprocess_image(new_image_path)

# Print the predicted green percentage
print(f"Detected Green View Index in your image is: {predicted_green_percentage:.2f}%")


Detected Green View Index in your image is: 54.94%


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
